In [18]:
import tensorflow as tf
import pandas as pd

In [13]:
dataset = tf.data.experimental.make_csv_dataset(
    '../../data/external/london_houses_data.csv',
    batch_size=32,
    num_epochs=1,
    label_name='history_price'
)

In [14]:
num_examples = sum(1 for _ in dataset)
part_size = num_examples // 7

2025-01-28 08:47:22.126291: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
parts = []

for i in range(7):
    start = i * part_size
    end = (i + 1) * part_size
    part = dataset.skip(start).take(end - start)
    parts.append(part)

In [19]:
for i, part in enumerate(parts):
    data = []
    for features in part:
        data.append(features)
    df = pd.DataFrame(data)


    df.to_csv(f"../../data/processed/train_data_{i}.csv", index=False)

2025-01-28 08:54:15.197073: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-01-28 08:54:50.892047: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [20]:
train_filepaths = '../../data/processed/train_data_*.csv'

In [21]:
filepath_dataset = tf.data.Dataset.list_files(
    train_filepaths,
    seed=42
)

In [22]:
filepath_dataset

<_ShuffleDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [23]:
for i in filepath_dataset:
    print(i)

tf.Tensor(b'../../data/processed/train_data_5.csv', shape=(), dtype=string)
tf.Tensor(b'../../data/processed/train_data_6.csv', shape=(), dtype=string)
tf.Tensor(b'../../data/processed/train_data_3.csv', shape=(), dtype=string)
tf.Tensor(b'../../data/processed/train_data_2.csv', shape=(), dtype=string)
tf.Tensor(b'../../data/processed/train_data_4.csv', shape=(), dtype=string)
tf.Tensor(b'../../data/processed/train_data_0.csv', shape=(), dtype=string)
tf.Tensor(b'../../data/processed/train_data_1.csv', shape=(), dtype=string)


In [24]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers
)

In [25]:
for line in dataset.take(5):
    print(line.numpy())

b'"OrderedDict([(\'fullAddress\', <tf.Tensor: shape=(32,), dtype=string, numpy='
b'"OrderedDict([(\'fullAddress\', <tf.Tensor: shape=(32,), dtype=string, numpy='
b'"OrderedDict([(\'fullAddress\', <tf.Tensor: shape=(32,), dtype=string, numpy='
b'"OrderedDict([(\'fullAddress\', <tf.Tensor: shape=(32,), dtype=string, numpy='
b'"OrderedDict([(\'fullAddress\', <tf.Tensor: shape=(32,), dtype=string, numpy='
